<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - ALS MRR</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als
from lenskit.algorithms.als import BiasedMF
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(features,reg):
    all_recs = []
    test_data = []
    
    features=int(features)
    als = BiasedMF(features=features, iterations=100, reg=reg, damping=5, bias=True,  progress=None) # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('ALS', als, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.recip_rank)
    results = rla.compute(all_recs, test_data)
                          
    return results["recip_rank"].mean()

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    features = params['features']
    reg = params['reg']
    metric = model_trainer(features,reg)
    print([features,reg,metric],',')
    return {'loss': -metric, 'status': STATUS_OK}

space={'features': hp.uniform('features', 1, 100),
       'reg': hp.uniform('reg',0.01,0.4)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[42.91131041193842, 0.2277745822036163, 0.20021561288594314]                                                           
,                                                                                                                      
[72.14031894981319, 0.3472479707651157, 0.2043377323726451]                                                            
,                                                                                                                      
[47.81954333597293, 0.24850165085796644, 0.20329358109449447]                                                          
,                                                                                                                      
[85.14868042112461, 0.014414233861067629, 0.21664789300631798]                                                         
,                                                                                                                      
[21.949450326709066, 0.16380264465889746

[74.3580306886833, 0.27235878276865205, 0.20176250727670025]                                                           
,                                                                                                                      
[60.43549763148129, 0.24307721978493801, 0.22090204925965945]                                                          
,                                                                                                                      
[54.86985155530731, 0.23932377972166938, 0.20417885452585555]                                                          
,                                                                                                                      
[42.04178756231505, 0.25480786261162025, 0.20935460108981024]                                                          
,                                                                                                                      
[61.27110609744065, 0.21442968194061202,

[87.36827042643404, 0.17546606398790277, 0.198428689452987]                                                            
,                                                                                                                      
[57.5168279612191, 0.39882698946866557, 0.20580408468106307]                                                           
,                                                                                                                      
[91.56298651684233, 0.26822929811710855, 0.1967695778980952]                                                           
,                                                                                                                      
[74.05619447673206, 0.22549924916286143, 0.22361282283561115]                                                          
,                                                                                                                      
[78.12595313490434, 0.1338989883520716, 

In [5]:
metric = pd.DataFrame(array)
metric.columns = ['features','reg','MRR']
metric.head(5)

,features,reg,MRR
0,42.911310,0.227775,0.200216
1,72.140319,0.347248,0.204338
2,47.819543,0.248502,0.203294
3,85.148680,0.014414,0.216648
4,21.949450,0.163803,0.182231


In [6]:
metric.sort_values(by=['MRR'],ascending=False).head(10)

,features,reg,MRR
71,74.056194,0.225499,0.223613
66,94.231517,0.398401,0.221618
78,87.396130,0.071076,0.221400
33,75.260371,0.234304,0.221083
35,60.435498,0.243077,0.220902
67,97.597804,0.313128,0.220623
10,94.342978,0.316739,0.219555
39,55.986971,0.178864,0.219491
20,88.007180,0.134034,0.219405
72,78.125953,0.133899,0.218130


In [7]:
metric.to_csv('metric_MRR.csv', index=False)

In [15]:
%matplotlib qt
import matplotlib.pyplot as plt
from matplotlib import cm
colormap = cm.hot
x = metric['reg']
y = metric['features'] 
c = metric['MRR']


fig = plt.figure()
img = plt.scatter(x, y, c=c, cmap=plt.jet())
plt.xlabel('reg')
plt.ylabel('features')
fig.colorbar(img)
plt.show()